# Data exploration

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt

C:\Users\mirei\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\mirei\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\Users\mirei\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)
C:\Users\mirei\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


### Import data

In [2]:
nodes_hd = pd.read_csv('final data/graph_nodes_v2024-05-27.csv') #04-02
edges_hd = pd.read_csv("final data/graph_edges_v2024-05-27.csv") #04-02
print(len(nodes_hd))
len(edges_hd)

17083


C:\Users\mirei\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3417: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


1356448

In [3]:
nodes_ad = pd.read_csv('final data/alz_graph_nodes_v2024-05-25.csv')#"ALZHEIMER/monarch_nodes_v2024-05-25.csv")
edges_ad = pd.read_csv('final data/alz_graph_edges_v2024-05-25.csv')#"ALZHEIMER/monarch_edges_v2024-05-25.csv")
print(len(nodes_ad))
len(edges_ad)

17685


1450686

There is more than one kind of Alzheimer's

In [4]:
alzh_ids = nodes_ad[(nodes_ad.preflabel.str.contains('Alzheimer')) & (nodes_ad.semantic_groups=='DISO')]
print(len(alzh_ids))

24


### Cleaning + Distribution relationships and semantic groups

In [5]:
edges_hd['property_label'] = edges_hd['property_label'].fillna('NA')
edges_hd['property_label'] = edges_hd['property_label'].str.replace('biolink:', '')
edges_hd['property_label'] = edges_hd['property_label'].str.replace('_', ' ')
print("Relationships in the Huntington's graph")
print(edges_hd['property_label'].value_counts())

Relationships in the Huntington's graph


In [6]:
edges_ad['property_label'] = edges_ad['property_label'].fillna('NA')
edges_ad['property_label'] = edges_ad['property_label'].str.replace('biolink:', '')
edges_ad['property_label'] = edges_ad['property_label'].str.replace('_', ' ')
print("Relationships in the Alzheimer's graph")
print(edges_ad['property_label'].value_counts())

Relationships in the Alzheimer's graph


In [7]:
nodes_ad['semantic_groups'] = nodes_ad['semantic_groups'].fillna('NA')
# in case it is in the other graph
na = nodes_ad[nodes_ad['semantic_groups'] == 'NA'].id
for node in na:
    nodes_ad.loc[nodes_ad.id == node, 'semantic_groups'] = list(nodes_hd[nodes_hd.id == node]['semantic_groups'])[0]

nodes_ad['semantic_groups'] = nodes_ad['semantic_groups'].fillna('NA')
   
print(nodes_ad['semantic_groups'].value_counts())

In [8]:
nodes_hd['semantic_groups'] = nodes_hd['semantic_groups'].fillna('NA')
# in case it is in the other graph
na = nodes_hd[nodes_hd['semantic_groups'] == 'NA'].id
for node in na:
    nodes_hd.loc[nodes_hd.id == node, 'semantic_groups'] = list(nodes_ad[nodes_ad.id == node]['semantic_groups'])[0]
    
print(nodes_hd['semantic_groups'].value_counts())

### Transcriptomics post

## Iron

In [18]:
desc = nodes_hd.dropna(subset=['description']) 
iron = desc[desc.description.str.contains('iron')]

name = nodes_hd.dropna(subset=['name']) 
iron2 = name[name.name.str.contains('iron')]
print(len(iron), len(iron2))
iron_t = pd.concat([iron, iron2]).drop_duplicates(keep=False)
len(iron_t)

136 26


120

In [ ]:
# Create a set of already connected pairs
connected_pairs = set(zip(edges_hd['subject_id'], edges_hd['object_id'])) | set(zip(edges_ad['subject_id'], edges_ad['object_id']))

# Get all pairs that are not already connected
pairs_i = []
ids = set(iron_t['id'])

for i, id_i in enumerate(ids):
    for id_j in list(ids)[i+1:]:
        if (id_i, id_j) not in connected_pairs and (id_j, id_i) not in connected_pairs:
            pairs_i.append((id_i, id_j))

print(len(pairs_i))

In [26]:
# get all pairs that are not already connected
ids = list(iron_t.id)

pairs_i = []
for i in range(len(ids)):
    for j in range(len(ids)):
        if i < j:
            if len(edges_hd[(edges_hd.subject_id == ids[i]) & (edges_hd.object_id == ids[j])]) == 0:
                if len(edges_hd[(edges_hd.subject_id == ids[j]) & (edges_hd.object_id == ids[i])]) == 0:
                    if len(edges_ad[(edges_ad.subject_id == ids[i]) & (edges_ad.object_id == ids[j])]) == 0:
                        if len(edges_ad[(edges_ad.subject_id == ids[j]) & (edges_ad.object_id == ids[i])]) == 0:
                            pairs_i.append((ids[i], ids[j]))

print(len(pairs_i))

6933


In [22]:
pairs_pred_iron = pd.DataFrame(pairs, columns=['subject_id', 'object_id'])
# pairs_pred.info()
pairs_pred_iron.to_csv('pairs_pred_iron.csv', index=False)  

In [23]:
pairs_pred_iron.head()

,subject_id,object_id
0,HGNC:48,HGNC:7715
1,HGNC:48,HGNC:7708
2,HGNC:48,HGNC:728
3,HGNC:48,HGNC:12876
4,HGNC:48,HGNC:7707


## Metrics

In [12]:
def create_graph(df, source, target):
    G = nx.DiGraph()
    # Add nodes from the 'source' and 'target' columns
    G.add_nodes_from(df[source])
    G.add_nodes_from(df[target])
    # Add edges from the DataFrame
    edges = [(row[source], row[target]) for index, row in df.iterrows()]
    G.add_edges_from(edges)
    return G

In [10]:
ad_graph = create_graph(edges_ad, 'subject_id', 'object_id')

In [13]:
hd_graph = create_graph(edges_hd, 'subject_id', 'object_id')

In [14]:
def degree(graph, nodes, top, plot=False, title="Huntington's"):
    
    df = pd.DataFrame()
#     print('degree absolute')
    deg = graph.degree()
    deg_s = dict(sorted(deg, key=lambda x:x[1], reverse=True)[:top])
    
#     print('degree centrality - fraction')
    dc = nx.degree_centrality(graph)
    dc = dict(sorted(dc.items(), key=lambda x:x[1], reverse=True)[:top])
 
    for node_id in dc:
        preflabel = nodes.loc[nodes['id'] == node_id, 'preflabel'].values[0] if node_id in nodes['id'].values else None        
        df = df.append({'id': node_id,'preflabel': preflabel,'degree centrality': dc[node_id],'degree': deg[node_id] }, ignore_index=True)
        
    # plot
    if plot == True:
        degrees = dict(deg)
        pos_degree_vals = list(filter(lambda val: val > 0, degrees.values())) # filtering nodes outdegree values with outdegree > 0
        uq_pos_degree_vals = sorted(set(pos_degree_vals)) # getting unique and sorted outdegree values
        hist = [pos_degree_vals.count(x) for x in uq_pos_degree_vals] # counting frequency of each outdegree values

        x = np.asarray(uq_pos_degree_vals, dtype = float)
        y = np.asarray(hist, dtype = float)

        logx = np.log10(x)
        logy = np.log10(y)

        plt.figure(figsize=(10,10))
        plt.xlim(min(logx), max(logx))
        plt.xlabel('log10 (Degree)')
        plt.ylabel('log10 (Number of nodes)')
        plt.title("Degree Distribution of"+title)
        degree_dist = plt.plot(logx, logy, 'o')

    return df

In [16]:
metrics_hd = degree(hd_graph, nodes_hd, 5, plot=False, title="Huntington's")
metrics_hd

,id,preflabel,degree centrality,degree
0,GO:0005515,protein binding,0.643016,10984.0
1,HGNC:7553,MYC,0.294345,5028.0
2,HGNC:31859,TRIM67,0.259806,4438.0
3,HGNC:9514,PSG1,0.226203,3864.0
4,HGNC:18224,ZRANB1,0.225442,3851.0


In [12]:
# metrics_ad = pd.DataFrame()
metrics_ad = degree(ad_graph, nodes_ad, 5, plot=False, title="Alzheimer's")
metrics_ad

,id,preflabel,degree centrality,degree
0,GO:0005515,protein binding,0.623388,11024.0
1,HGNC:7553,MYC,0.284494,5031.0
2,GO:0005634,nucleus,0.271149,4795.0
3,GO:0005829,cytosol,0.270414,4782.0
4,HGNC:31859,TRIM67,0.251640,4450.0


In [31]:
# higher Closeness Centrality score indicates more central
def closeness_centrality(graph, nodes):
    df = pd.DataFrame()
    clos = nx.closeness_centrality(graph)
    close = dict(sorted(clos.items(), key=lambda x:x[1], reverse=True)[:5])
    for node_id in close:
        preflabel = nodes.loc[nodes['id'] == node_id, 'preflabel'].values[0] if node_id in nodes['id'].values else None        
        df = df.append({'id': node_id,'preflabel': preflabel,'closeness': close[node_id] }, ignore_index=True)
        
    return df

In [32]:
closeness = closeness_centrality(ad_graph, nodes_ad)
closeness

,id,preflabel,closeness
0,GO:0005515,protein binding,0.672139
1,GO:0005829,cytosol,0.488325
2,GO:0005737,cytoplasm,0.487895
3,GO:0005634,nucleus,0.486638
4,GO:0005886,plasma membrane,0.464297


In [33]:
closeness = closeness_centrality(hd_graph, nodes_hd)
closeness

,id,preflabel,closeness
0,GO:0005515,protein binding,0.692476
1,HP:0000006,Autosomal dominant inheritance,0.436201
2,HGNC:620,APP,0.433730
3,UBERON:0003701,calcaneal tendon,0.432212
4,UBERON:0003053,ventricular zone,0.428620


In [34]:
# higher betweenness Centrality score indicates more central
def betweenness_centrality(graph, nodes, top=5):
    df = pd.DataFrame()
    betw = nx.betweenness_centrality(hd_graph)
    betweenness = dict(sorted(betw.items(), key=lambda x:x[1])[:top])
    for node_id in close:
        preflabel = nodes.loc[nodes['id'] == node_id, 'preflabel'].values[0] if node_id in nodes['id'].values else None        
        df = df.append({'id': node_id,'preflabel': preflabel,'betweenness': betweenness[node_id] }, ignore_index=True)
        
    return df

In [35]:
betweenness = betweenness_centrality(ad_graph, nodes_ad, 5)
betweenness

NameError: name 'close' is not defined

In [ ]:
betweenness = betweenness_centrality(hd_graph, nodes_hd, 5)
betweenness

## Finding commonalities

### Common nodes

In [9]:
total_r = len(list(nodes_ad.id)+ list(nodes_hd.id))
total = len(set(list(nodes_ad.id)+ list(nodes_hd.id)))
common = (total_r - total)
print('Total with repe:', total_r)
print('Total wo repe:', total)
print('In common ', common)
dis_hd = len(nodes_hd.id) - common
dis_ad = len(nodes_ad.id) - common
print('Distinct HD', dis_hd)
print('Distinct AD', dis_ad)

Total with repe: 34768
Total wo repe: 17772
In common  16996
Distinct HD 87
Distinct AD 689


In [10]:
nodes_f = pd.merge(nodes_ad, nodes_hd, on=['id'], how='inner', indicator=False)
print(len(nodes_f) == common)
# to correctly check the semantic groups
nodes = nodes_hd[nodes_hd.id.isin(nodes_f.id)]
print(nodes['semantic_groups'].value_counts())

True
GENE    16193
NA        770
DISO       26
ANAT        4
GENO        2
PHYS        1
Name: semantic_groups, dtype: int64


### Common edges

In [ ]:
hd = edges_hd.drop_duplicates()
ad = edges_ad.drop_duplicates()
hd['subject_id'] = hd['subject_id'].astype(str)
hd['object_id'] = hd['object_id'].astype(str)
hd['property_id'] = hd['property_id'].astype(str)

ad['subject_id'] = ad['subject_id'].astype(str)
ad['object_id'] = ad['object_id'].astype(str)
ad['property_id'] = ad['property_id'].astype(str)
print(len(hd), len(ad))

common_rows = hd.apply(tuple, 1).isin(ad.apply(tuple, 1))
hd_distinct = hd[~hd.apply(tuple,1).isin(ad.apply(tuple,1))]
ad_distinct = ad[~ad.apply(tuple,1).isin(hd.apply(tuple,1))]
common = len(common_rows[common_rows == True])
print('In common:', common)
print('Only in HD:', len(hd_distinct), common+len(hd_distinct) == len(hd))
print('Only in AD:',len(ad_distinct), common+len(ad_distinct) == len(ad))

In [ ]:
cc = hd[common_rows]
cc['property_label'].value_counts()

In [34]:
ad_distinct.to_csv('ad_edges.csv', index=False)

### TO predict

In [14]:
nodes_predict2 = nodes_hd[~nodes_hd.id.isin(nodes_f.id)]
nodes_predict2.semantic_groups.value_counts()

DISO    52
GENE    20
ANAT     6
GENO     6
PHYS     3
Name: semantic_groups, dtype: int64

In [15]:
nodes_predict2[nodes_predict2.preflabel.str.contains('Hunt')]

,id,semantic_groups,preflabel,synonyms,name,description
11743,MONDO:0016621,DISO,juvenile Huntington disease,NaN,juvenile Huntington disease,NaN
11937,MONDO:0007739,DISO,Huntington disease,NaN,Huntington disease,NaN
16141,MONDO:0000167,DISO,Huntington disease and related disorders,NaN,Huntington disease and related disorders,NaN


In [ ]:
# nodes only in HD graph
nodes_predict2 = nodes_hd[~nodes_hd.id.isin(nodes_f.id)]
ids = list(nodes_predict2.id)

# get all pairs that are not already connected
pairs = []
for i in range(len(ids)):
    for j in range(len(ids)):
        if i < j:
            if len(edges_hd[(edges_hd.subject_id == ids[i]) & (edges_hd.object_id == ids[j])]) == 0:
                if len(edges_hd[(edges_hd.subject_id == ids[j]) & (edges_hd.object_id == ids[i])]) == 0:
                    pairs.append((ids[i], ids[j]))

print(len(pairs))

In [33]:
pairs_pred = pd.DataFrame(pairs, columns=['subject_id', 'object_id'])
# pairs_pred.info()
pairs_pred.to_csv('pairs_pred.csv', index=False)  

### Common phenotypes

In [37]:
# FIND COMMON PHENOTYPES

# phenotypes directly attached to HD
pheno_h = edges_hd[(edges_hd['property_label'] == 'has phenotype') & (edges_hd['subject_id'] == 'MONDO:0007739')]
phenotypes_h = nodes_hd[nodes_hd['id'].isin(list(pheno_h.object_id))]
# phenotypes_h['preflabel'] = phenotypes_h['preflabel'].apply(lambda x: x.lower() if isinstance(x, str) else x)
print(len(phenotypes_h))

# phenotypes directly attached to AD
pheno_a = edges_ad[(edges_ad['property_label'] == 'has phenotype') & (edges_ad['subject_id'].isin(list(alzh_ids.id)))]
phenotypes_a = nodes_ad[nodes_ad['id'].isin(list(pheno_a.object_id))]
# phenotypes_a['preflabel'] = phenotypes_a['preflabel'].apply(lambda x: x.lower() if isinstance(x, str) else x)
print(len(phenotypes_a))

# both have the same phenotypes
phenotypes = pd.merge(phenotypes_a, phenotypes_h, on=['id'], how='inner', indicator=True)
print(list(phenotypes.preflabel_x))

59
47
['Seizure', 'Agitation', 'Gait disturbance', 'Dystonia', 'Babinski sign', 'Memory impairment', 'Myoclonus', 'Dementia', 'Hallucinations', 'Disinhibition', 'Personality changes']


### Different edges

### Common Treatments

In [34]:
# treats = 'treats or applied or studied to treat'
drugs_ad = edges_ad[(edges_ad.property_id.str.contains('treats')) & (edges_ad.object_id.isin(list(alzh_ids.id)))]
drugs = nodes_ad[nodes_ad.id.isin(list(drugs_ad.subject_id))]
# drugs

In [35]:
drugs_hd = edges_hd[(edges_hd.property_id.str.contains('treats'))& (edges_hd.object_id == 'MONDO:0007739')]
drugs_hd = nodes_hd[nodes_hd.id.isin(list(drugs_hd.subject_id))]
# drugs_hd

In [36]:
skip = (set(list(drugs.id)).intersection(list(drugs_hd.id)))
drugs_predict = pd.DataFrame()
for drug in list(drugs.id):
    if drug not in skip:
        drugs_predict = drugs_predict.append({'subject_id': drug, 'object_id': 'MONDO:0007739'}, ignore_index=True)
drugs_predict.to_csv('pairs_pred_drugs.csv', index=False)

### Common genes -- none

In [25]:
hd_conn = edges_hd[(edges_hd.subject_id == 'MONDO:0007739')|(edges_hd.object_id == 'MONDO:0007739')]
print(len(hd_conn))
# get rid of things we have explored already
hd_conn = hd_conn[(hd_conn.property_id != 'biolink:has_phenotype') & (hd_conn.property_id != 'biolink:treats_or_applied_or_studied_to_treat')]
a = nodes_hd[(nodes_hd.id.isin(list(hd_conn.subject_id))) | (nodes_hd.id.isin(list(hd_conn.object_id)))]
nodes_ad[nodes_ad.id.isin(list(a.id))]

78


,id,semantic_groups,preflabel,synonyms,name,description
763,HP:0000006,DISO,Autosomal dominant inheritance,NaN,Autosomal dominant inheritance,NaN
4333,HGNC:11007,GENE,SLC2A3,GLUT3,solute carrier family 2 member 3,Enables dehydroascorbic acid transmembrane tra...
5621,HGNC:4851,GENE,HTT,5-HTT|5-HTTLPR|5HTT|HTT|OCD1|SERT|SERT1|hSERT,solute carrier family 6 member 4,This gene encodes an integral membrane protein...
9907,MONDO:0001627,DISO,dementia,NaN,dementia,NaN
12280,HP:0000657,DISO,Oculomotor apraxia,NaN,Oculomotor apraxia,NaN


In [31]:
ad_conn = edges_ad[(edges_ad['object_id'].isin(list(alzh_ids.id))) | (edges_ad['subject_id'].isin(list(alzh_ids.id)))]
print(len(ad_conn))
# get rid of things we have explored already
ad_conn = ad_conn[(ad_conn.property_id != 'biolink:has_phenotype') & (ad_conn.property_id != 'biolink:treats_or_applied_or_studied_to_treat')]
b = nodes_ad[(nodes_ad.id.isin(list(ad_conn.subject_id))) | (nodes_ad.id.isin(list(ad_conn.object_id)))]
set(list(a.id)).intersection(list(b.id))

171


{'HP:0000006', 'MONDO:0001627'}

In [28]:
htt_a = edges_ad[(edges_ad.subject_id=='HGNC:4851')|(edges_ad.object_id=='HGNC:4851')]
htt_a[(htt_a.property_label != 'interacts with') & (htt_a.property_label != 'has phenotype')]

,subject_id,property_id,object_id,reference_uri,reference_supporting_text,reference_date,property_label,property_description,property_uri
56558,HGNC:4851,biolink:expressed_in,UBERON:0003701,NaN,This edge comes from the Monarch Knowledge Gra...,NaN,expressed in,NaN,http://purl.obolibrary.org/obo/biolink_express...
59408,HGNC:4851,biolink:expressed_in,UBERON:0001150,NaN,This edge comes from the Monarch Knowledge Gra...,NaN,expressed in,NaN,http://purl.obolibrary.org/obo/biolink_express...
72972,HGNC:4851,biolink:expressed_in,CL:0002255,NaN,This edge comes from the Monarch Knowledge Gra...,NaN,expressed in,NaN,http://purl.obolibrary.org/obo/biolink_express...
80813,HGNC:4851,biolink:located_in,GO:0030425,NaN,This edge comes from the Monarch Knowledge Gra...,NaN,located in,NaN,http://purl.obolibrary.org/obo/biolink_located_in
492960,HGNC:4851,biolink:located_in,GO:0048471,NaN,This edge comes from the Monarch Knowledge Gra...,NaN,located in,NaN,http://purl.obolibrary.org/obo/biolink_located_in
662414,HGNC:4851,biolink:actively_involved_in,GO:0043065,NaN,This edge comes from the Monarch Knowledge Gra...,NaN,actively involved in,NaN,http://purl.obolibrary.org/obo/biolink_activel...
813399,HGNC:4851,biolink:located_in,GO:0005770,NaN,This edge comes from the Monarch Knowledge Gra...,NaN,located in,NaN,http://purl.obolibrary.org/obo/biolink_located_in
833321,HGNC:4851,biolink:located_in,GO:0005769,NaN,This edge comes from the Monarch Knowledge Gra...,NaN,located in,NaN,http://purl.obolibrary.org/obo/biolink_located_in
880780,HGNC:4851,biolink:enables,GO:0005515,NaN,This edge comes from the Monarch Knowledge Gra...,NaN,enables,NaN,http://purl.obolibrary.org/obo/biolink_enables
1050518,HGNC:4851,biolink:enables,GO:0034452,NaN,This edge comes from the Monarch Knowledge Gra...,NaN,enables,NaN,http://purl.obolibrary.org/obo/biolink_enables


In [40]:
edges_hd[edges_hd.subject_id == 'CHEBI:50694']

,subject_id,property_id,object_id,reference_uri,reference_supporting_text,reference_date,property_label,property_description,property_uri
229079,CHEBI:50694,biolink:treats_or_applied_or_studied_to_treat,MONDO:0005395,https://www.ncbi.nlm.nih.gov/pubmed/21561882,This edge comes from the Monarch Knowledge Gra...,NaN,treats or applied or studied to treat,NaN,http://purl.obolibrary.org/obo/biolink_treats_...
717031,CHEBI:50694,biolink:treats_or_applied_or_studied_to_treat,MONDO:0007739,"https://www.ncbi.nlm.nih.gov/pubmed/11711215,1...",This edge comes from the Monarch Knowledge Gra...,NaN,treats or applied or studied to treat,NaN,http://purl.obolibrary.org/obo/biolink_treats_...
